In [48]:
import datetime
HEADER="""DELETE FROM public."User"; INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
"""
ROW="( {user_id}, '{name}', '{birth_date}','{gender}', '{from_date}', '{to_date}', {location_id}, {segment_id} )"
RECORDS=1000
LOCATION_RANGE=(1,17) # 17 locations
SEGMENT_RANGE=(1,3) # 3 segments
FUTURE=datetime.date(2999,12,31)
GENDERS=["M","F","O"]
BIRTH_DATE_START=datetime.date(1950,1,1)
BIRTH_DATE_RANGE=365*55 # 55 years
SEGMENT_AGES_RANGES=[(0,25),(25,45),(45,100)] # 3 segments
INSERT_USERS_FILE="../15_insert_users.sql"

In [49]:
import faker
import random

faker_name = faker.Faker()
today = datetime.date.today()
insert_users = []
for i in range(1,RECORDS+1):
    name = faker_name.name()
    
    birth_date = BIRTH_DATE_START + datetime.timedelta(days=random.randint(0,BIRTH_DATE_RANGE))
    from_date = today - datetime.timedelta(days=random.randint(180,365*5))
    location_id = random.randint(*LOCATION_RANGE)
    gender = random.choice(GENDERS)
    age = (today - birth_date).days // 365
    age_segment = list(filter(lambda x : x[0] <= age < x[1], SEGMENT_AGES_RANGES))
    segment_id = random.randint(*SEGMENT_RANGE) + SEGMENT_AGES_RANGES.index(age_segment[0]) * 3
    insert_users.append(
        dict(
            user_id=i,
            name= name,
            birth_date= birth_date,
            from_date= from_date,
            to_date= FUTURE,
            gender=gender,
            location_id=location_id,
            segment_id=segment_id
        )
    )
    print(ROW.format(**insert_users[-1]))



( 1, 'Daniel Johnson', '1993-01-08','O', '2022-05-24', '2999-12-31', 12, 5 )
( 2, 'Kendra Watkins', '1968-07-25','M', '2021-07-19', '2999-12-31', 14, 9 )
( 3, 'Michael Watson', '1983-02-01','F', '2022-01-31', '2999-12-31', 15, 6 )
( 4, 'Brian Stephenson', '1988-11-28','F', '2019-03-25', '2999-12-31', 2, 5 )
( 5, 'Melissa Thomas', '1955-12-21','O', '2022-08-07', '2999-12-31', 9, 7 )
( 6, 'Deanna Collier', '1994-04-13','M', '2019-01-10', '2999-12-31', 10, 6 )
( 7, 'Lawrence Hodges', '1965-04-05','O', '2021-08-06', '2999-12-31', 14, 7 )
( 8, 'Amy Delgado', '1993-09-17','M', '2019-01-31', '2999-12-31', 15, 4 )
( 9, 'Chad Stevenson', '1983-12-22','F', '2021-05-08', '2999-12-31', 5, 4 )
( 10, 'Christopher Gill', '1971-08-10','F', '2022-10-31', '2999-12-31', 1, 9 )
( 11, 'John Houston', '1977-08-04','O', '2021-12-10', '2999-12-31', 7, 7 )
( 12, 'Steven Walker', '1958-05-12','M', '2021-08-30', '2999-12-31', 14, 7 )
( 13, 'Raymond Kirby', '1957-11-28','F', '2021-01-04', '2999-12-31', 6, 9 )
( 1

In [50]:
## MAKE INSERT STATEMENT
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**user) for user in insert_users])
insert_sql += ';'
print(insert_sql)


DELETE FROM public."User"; INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
( 1, 'Daniel Johnson', '1993-01-08','O', '2022-05-24', '2999-12-31', 12, 5 ),
( 2, 'Kendra Watkins', '1968-07-25','M', '2021-07-19', '2999-12-31', 14, 9 ),
( 3, 'Michael Watson', '1983-02-01','F', '2022-01-31', '2999-12-31', 15, 6 ),
( 4, 'Brian Stephenson', '1988-11-28','F', '2019-03-25', '2999-12-31', 2, 5 ),
( 5, 'Melissa Thomas', '1955-12-21','O', '2022-08-07', '2999-12-31', 9, 7 ),
( 6, 'Deanna Collier', '1994-04-13','M', '2019-01-10', '2999-12-31', 10, 6 ),
( 7, 'Lawrence Hodges', '1965-04-05','O', '2021-08-06', '2999-12-31', 14, 7 ),
( 8, 'Amy Delgado', '1993-09-17','M', '2019-01-31', '2999-12-31', 15, 4 ),
( 9, 'Chad Stevenson', '1983-12-22','F', '2021-05-08', '2999-12-31', 5, 4 ),
( 10, 'Christopher Gill', '1971-08-10','F', '2022-10-31', '2999-12-31', 1, 9 ),
( 11, 'John Houston', '1977-08-04','O', '2021-12-10', '2999-12-31', 7, 

In [51]:
with open(INSERT_USERS_FILE, 'w') as f:
    f.write(insert_sql)

In [52]:
#### Set user subscriptions
HEADER="""DELETE FROM public."UserSubscription"; INSERT INTO public."UserSubscription"(
    "UserId", "SubscriptionId", "ValidFrom", "ValidTo") VALUES
"""
ROW="( {user_id}, {subscription_id}, '{from_date}', '{to_date}' )"
SUBSCRIPTIONS_RANGE=(1,3) # 3 subscriptions
CHANGE_SUB_ONE_TIME=0.5
CHANGE_SUB_TWO_TIMES=0.3

In [53]:
# Set initial subs for every user, sharing same start date
insert_subscriptions = []
for user in insert_users:
    insert_subscriptions.append(
        dict(
            user_id=user['user_id'],
            subscription_id=random.randint(*SUBSCRIPTIONS_RANGE),
            from_date=user['from_date'],
            to_date=user['from_date'] + datetime.timedelta(days=round((datetime.date.today() - user['from_date']).days * random.random()))
        )
    )
    print(ROW.format(**insert_subscriptions[-1]))

( 1, 2, '2022-05-24', '2023-05-09' )
( 2, 3, '2021-07-19', '2023-05-11' )
( 3, 3, '2022-01-31', '2023-06-25' )
( 4, 3, '2019-03-25', '2019-07-22' )
( 5, 1, '2022-08-07', '2023-11-16' )
( 6, 3, '2019-01-10', '2023-04-15' )
( 7, 1, '2021-08-06', '2023-08-18' )
( 8, 1, '2019-01-31', '2022-07-15' )
( 9, 2, '2021-05-08', '2022-11-02' )
( 10, 2, '2022-10-31', '2023-09-18' )
( 11, 3, '2021-12-10', '2023-02-13' )
( 12, 1, '2021-08-30', '2021-11-17' )
( 13, 1, '2021-01-04', '2021-08-03' )
( 14, 1, '2022-04-27', '2023-04-29' )
( 15, 2, '2021-09-19', '2021-09-20' )
( 16, 3, '2022-06-20', '2022-06-23' )
( 17, 2, '2022-09-15', '2023-02-05' )
( 18, 1, '2020-07-27', '2021-06-01' )
( 19, 1, '2022-10-06', '2023-05-29' )
( 20, 1, '2019-08-28', '2020-05-07' )
( 21, 3, '2019-07-11', '2021-10-14' )
( 22, 2, '2019-04-03', '2023-08-02' )
( 23, 2, '2022-08-14', '2022-09-09' )
( 24, 3, '2020-03-18', '2021-12-02' )
( 25, 3, '2023-01-06', '2023-11-28' )
( 26, 2, '2021-02-05', '2022-08-15' )
( 27, 2, '2020-11-07'

In [54]:
lost_customers_index = random.sample(
    range(0,len(insert_subscriptions)), 
    int((CHANGE_SUB_ONE_TIME+CHANGE_SUB_TWO_TIMES)*len(insert_subscriptions))
)
len(lost_customers_index)


800

In [55]:
# Change subscription once
from collections import defaultdict


cut_index = int(CHANGE_SUB_ONE_TIME*len(insert_users))
for i in lost_customers_index:
    new_sub = insert_subscriptions[i]['subscription_id']
    while insert_subscriptions[i]['subscription_id'] == (new_sub := random.randint(*SUBSCRIPTIONS_RANGE)): ... 
    insert_subscriptions.append(
        dict(
            user_id=insert_subscriptions[i]['user_id'],
            subscription_id=new_sub ,
            from_date=insert_subscriptions[i]['to_date'] + datetime.timedelta(days=1),
            to_date=FUTURE
        )
    )
# Change subscription twice    
for i in lost_customers_index[cut_index:]:
    # Find a new subscription change date
    days_first_sub = int((today - insert_subscriptions[i]['to_date']).days * random.random())
    # Change to date from previous subscription
    insert_subscriptions[i]['to_date'] = insert_subscriptions[i]['from_date'] + datetime.timedelta(days=days_first_sub)
    # Add new subscription
    while insert_subscriptions[i]['subscription_id'] == (new_sub := random.randint(*SUBSCRIPTIONS_RANGE)): ... 
    insert_subscriptions.append(
        dict(
            user_id=insert_subscriptions[i]['user_id'],
            subscription_id=new_sub ,
            from_date=insert_subscriptions[i]['to_date'] + datetime.timedelta(days=1),
            to_date=FUTURE
        )
    )

  

In [56]:
# Check users with miltiple rows
repeated_user_id_count = defaultdict(int)
for item in insert_subscriptions:
    print(repeated_user_id_count[item['user_id']])
    repeated_user_id_count[item['user_id']] += 1
count = [0,0,0]
for v in repeated_user_id_count.values():
    count[v-1] += 1
print(f'Users with 1 subscription: {count[0]}')
print(f'Users with 2 subscriptions: {count[1]}')
print(f'Users with 3 subscriptions: {count[2]}')  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [57]:
len(insert_subscriptions)

2100

In [58]:
## Create user subscriptions insert statement
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**sub) for sub in insert_subscriptions])
insert_sql += ';'   

print(insert_sql)

DELETE FROM public."UserSubscription"; INSERT INTO public."UserSubscription"(
    "UserId", "SubscriptionId", "ValidFrom", "ValidTo") VALUES
( 1, 2, '2022-05-24', '2023-05-09' ),
( 2, 3, '2021-07-19', '2021-07-28' ),
( 3, 3, '2022-01-31', '2022-03-04' ),
( 4, 3, '2019-03-25', '2019-07-22' ),
( 5, 1, '2022-08-07', '2023-11-16' ),
( 6, 3, '2019-01-10', '2023-04-15' ),
( 7, 1, '2021-08-06', '2023-08-18' ),
( 8, 1, '2019-01-31', '2020-06-27' ),
( 9, 2, '2021-05-08', '2021-12-16' ),
( 10, 2, '2022-10-31', '2022-11-12' ),
( 11, 3, '2021-12-10', '2023-02-13' ),
( 12, 1, '2021-08-30', '2021-11-17' ),
( 13, 1, '2021-01-04', '2021-08-03' ),
( 14, 1, '2022-04-27', '2023-04-29' ),
( 15, 2, '2021-09-19', '2022-08-28' ),
( 16, 3, '2022-06-20', '2022-11-30' ),
( 17, 2, '2022-09-15', '2023-02-05' ),
( 18, 1, '2020-07-27', '2021-06-01' ),
( 19, 1, '2022-10-06', '2023-05-29' ),
( 20, 1, '2019-08-28', '2019-12-25' ),
( 21, 3, '2019-07-11', '2021-10-14' ),
( 22, 2, '2019-04-03', '2023-08-02' ),
( 23, 2, '

In [59]:
# Write to file
INSERT_SUBSCRIPTIONS_FILE="../16_insert_user_subscriptions.sql"
with open(INSERT_SUBSCRIPTIONS_FILE, 'w') as f:
    f.write(insert_sql)